## This is my contribution to the IndabaX hackathon organized by IEEE sup'com in partnership with InstaDeep
# 
## The theme of the hackathon is "predicting the impact of mutation on protein stability"
### We want to predict the value of Delta Delta G (ddg) which is a metric for predicting how a single point mutation will affect protein stability.
### The features provided are the original protein sequence and its corresponding sequence after mutation as well as the exact amino acid that changed and the new amino acid in its place and finally the position of that mutation in the sequence

In [ ]:
### @title Download data from GCP bucket

import sys

if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

In [1]:
#@title Imports and moving to working directory

import torch 
import pandas as pd
from tqdm import tqdm
import numpy as np

# move to data folder
%cd indaba-data

[WinError 2] Le fichier spécifié est introuvable: 'indaba-data'
C:\Users\Ahmed Ben Aissa\Desktop\SUPCOM


In [3]:
df = pd.read_csv("train.csv")
df

,ID,pdb_id,mutation,wt_aa,mutation_pos,mut_aa,wt_seq,mut_seq,ddg
0,0,1GYZ,W1Q,W,1,Q,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,QIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.228775
1,1,1GYZ,W1E,W,1,E,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,EIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.496896
2,2,1GYZ,W1N,W,1,N,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,NIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.163002
3,3,1GYZ,W1H,W,1,H,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,HIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.209013
4,4,1GYZ,W1D,W,1,D,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,DIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.407602
...,...,...,...,...,...,...,...,...,...
339773,339773,r7-562-TrROS-Hall,V48W,V,48,W,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKW,-1.614409
339774,339774,r7-562-TrROS-Hall,V48Y,V,48,Y,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKY,-1.370771
339775,339775,r7-562-TrROS-Hall,V48F,V,48,F,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKF,-1.370829
339776,339776,r7-562-TrROS-Hall,V48P,V,48,P,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKP,-0.757622


#### We were provided with the embeddings of the sequences so here we are loading them for training along with the training and test sets

In [ ]:
#loading the training set and the embeddings of the 2 sequences (pre-mutation and post-mutation)
wt_emb = torch.load("/kaggle/working/indaba-data/train/train_wt.pt")
mut_emb = torch.load("/kaggle/working/indaba-data/train/train_mut.pt")
df = pd.read_csv("/kaggle/working/indaba-data/train/train.csv")

#creating a dataframe of the embeddings
wt = wt_emb.numpy()
wt = pd.DataFrame(wt)
del wt_emb

mut = mut_emb.numpy()
mut = pd.DataFrame(mut)
del mut_emb


# same for the test set
wt_embt = torch.load("/kaggle/working/indaba-data/test/test_wt.pt")
mut_embt = torch.load("/kaggle/working/indaba-data/test/test_mut.pt")
dft = pd.read_csv("/kaggle/working/indaba-data/test/test.csv")

wtt = wt_embt.numpy()
wtt = pd.DataFrame(wtt)
del wt_embt

mutt = mut_embt.numpy()
mutt = pd.DataFrame(mutt)


#### The embeddings dataframes have 1280 columns so I applied PCA to reduce that to 20

In [127]:
from sklearn.decomposition import PCA
import numpy as np


n_components = 20
pcaw = PCA(n_components=n_components)

# fit the PCA model to the embeddings
pcaw.fit(wt)

# transform the embeddings using the fitted PCA model
embw = pcaw.transform(wt)
embw = pd.DataFrame(embw, columns=['c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20'])

embwt = pcaw.transform(wtt)
embwt = pd.DataFrame(embwt, columns=['c1','c2','c3','c4','c5','c6','c7','c8','c9','c10','c11','c12','c13','c14','c15','c16','c17','c18','c19','c20'])


pcam = PCA(n_components=n_components)

# fit the PCA model to the embeddings
pcam.fit(mut)

# transform the embeddings using the fitted PCA model
embm = pcam.transform(mut)
embm = pd.DataFrame(embm,  columns=['c41','c42','c43','c44','c45','c46','c47','c48','c49','c50','c51','c52','c53','c54','c55','c56','c57','c58','c59','c60'])


embmt = pcam.transform(mutt)
embmt = pd.DataFrame(embmt,  columns=['c41','c42','c43','c44','c45','c46','c47','c48','c49','c50','c51','c52','c53','c54','c55','c56','c57','c58','c59','c60'])

#### creating a numerical feature for the position of the mutation

In [128]:
dft['mutation_pos'] = dft['mutation'].str.slice(1, -1).astype(int)

#### creating one big dataframe containing all the embeddings + the position

In [129]:
emb = pd.concat([embw, embm], axis=1)
emb.insert(0,'ddg',df['ddg'])
emb.insert(1,'mutation_pos',df['mutation_pos'])



embt = pd.concat([embwt, embmt], axis=1)
embt.insert(0,'mutation_pos',dft['mutation_pos'])
emb.columns

Index(['ddg', 'mutation_pos', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8',
       'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18',
       'c19', 'c20', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48',
       'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58',
       'c59', 'c60'],
      dtype='object')

In [154]:
emb

,ddg,mutation_pos,c1,c2,c3,c4,c5,c6,c7,c8,...,mut_aa_M,mut_aa_N,mut_aa_P,mut_aa_Q,mut_aa_R,mut_aa_S,mut_aa_T,mut_aa_V,mut_aa_W,mut_aa_Y
0,0.228775,1,-1.225055,-4.838012,-2.136732,-2.678357,-7.415438,0.517209,-0.513547,-4.651174,...,0,0,0,1,0,0,0,0,0,0
1,0.496896,1,-1.225055,-4.838012,-2.136732,-2.678357,-7.415438,0.517209,-0.513547,-4.651174,...,0,0,0,0,0,0,0,0,0,0
2,0.163002,1,-1.225055,-4.838012,-2.136732,-2.678357,-7.415438,0.517209,-0.513547,-4.651174,...,0,1,0,0,0,0,0,0,0,0
3,0.209013,1,-1.225055,-4.838012,-2.136732,-2.678357,-7.415438,0.517209,-0.513547,-4.651174,...,0,0,0,0,0,0,0,0,0,0
4,0.407602,1,-1.225055,-4.838012,-2.136732,-2.678357,-7.415438,0.517209,-0.513547,-4.651174,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339773,-1.614409,48,5.876492,12.370865,-4.072734,5.039557,2.111575,2.674126,-1.300102,2.263565,...,0,0,0,0,0,0,0,0,1,0
339774,-1.370771,48,5.876492,12.370865,-4.072734,5.039557,2.111575,2.674126,-1.300102,2.263565,...,0,0,0,0,0,0,0,0,0,1
339775,-1.370829,48,5.876492,12.370865,-4.072734,5.039557,2.111575,2.674126,-1.300102,2.263565,...,0,0,0,0,0,0,0,0,0,0
339776,-0.757622,48,5.876492,12.370865,-4.072734,5.039557,2.111575,2.674126,-1.300102,2.263565,...,0,0,1,0,0,0,0,0,0,0


In [35]:
del mut 
del wt

In [130]:
X = emb
Y = X.pop('ddg')#target column

### Scaling

In [112]:
from sklearn.preprocessing import StandardScaler

# Create an instance of StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data
scaler.fit(X)

# Transform the data
Xs = scaler.transform(X)

embt = scaler.transform(embt)

#### I used optuna to find the best hyperparameters for a CatBoostRegressor

In [ ]:
import optuna
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score

def objective(trial):
    
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "depth": trial.suggest_int("depth", 3, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-4, 10),
        "random_strength": trial.suggest_float("random_strength", 1e-4, 10),
        "task_type":"GPU"
    }
    cat = CatBoostRegressor(**params)
    
    return cross_val_score( cat, S, y, n_jobs=-1, cv=3,scoring='neg_mean_squared_error').mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=2000)

trial = study.best_trial
optuna.visualization.plot_optimization_history(study)
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

#### Now training the best CatBoost model found

In [ ]:
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
import numpy as np

cat = CatBoostRegressor(iterations=397, learning_rate=0.08, depth=9 , l2_leaf_reg=7,random_strength=0.8,task_type="GPU", random_state = 42)
cat.fit(X,Y)


## Prediction & submission

In [ ]:
del emb

In [132]:
y_pred = cat.predict(embt)

In [134]:
y_pred = pd.DataFrame({'ddg': y_pred})
y_pred.insert(0, 'ID', dft['ID'])

y_pred

,ID,ddg
0,0,-0.107781
1,1,-0.105348
2,2,-0.105468
3,3,-0.110506
4,4,-0.107080
...,...,...
1902,2408,-0.101583
1903,2409,-0.104860
1904,2410,-0.100849
1905,2411,-0.086464


In [141]:
y_pred.to_csv('submission.csv', index=False)